In [2]:
import json
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data_one = []
# data_two = []
# data_three = []
# data_four = []
reviewdata_df = []

In [4]:
data_file = open('F:\yelp\yelp_academic_dataset_review.json')
length = 2225213  # I calculated length prior to this calculation
for i, line in enumerate(data_file):
    if i<(length/4):
        data_one.append(json.loads(line))
data_file.close()

In [5]:
df = pd.DataFrame(data_one)

In [23]:
df.shape

(556303, 8)

In [6]:
data_main = []
with open('F:\yelp\yelp_academic_dataset_user.json') as f:
    for line in f:
        data_main.append(json.loads(line))

In [7]:
df_user = pd.DataFrame(data_main)

In [8]:
def user_with_friends(x):
    if len(x['friends']) != 0:
        return x

In [17]:
df_user_with_friends = df_user.apply(user_with_friends, axis=1)

In [18]:
# if len(yo_df['friends']) == 0:
#     print "yo"
df_user_with_friends = df_user_with_friends.dropna()


In [21]:
df_user.shape


(552339, 11)

In [22]:
df_user_with_friends.shape

(249440, 11)

In [24]:
merged_df = pd.merge(df, df_user_with_friends, on='user_id', how='inner')

In [25]:
merged_df.head(n=1)

,business_id,date,review_id,stars,text,type_x,user_id,votes_x,average_stars,compliments,elite,fans,friends,name,review_count,type_y,votes_y,yelping_since
0,UsFtqoBl7naz8AVUBZMjQQ,2013-11-08,Di3exaUCFNw1V4kSNW5pgA,5,All the food is great here. But the best thing...,review,uK8tzraOp4M5u3uYrqIBXg,"{u'funny': 0, u'useful': 0, u'cool': 0}",3.84,{},[],2,"[53wSMIztehsjP-vJZBS54A, CXhOU5apSRSTtEUiyoqH3Q]",Robin,46,user,"{u'funny': 2, u'useful': 36, u'cool': 5}",2013-05


In [26]:
merged_df.shape

(394815, 18)

In [27]:
merged_df.stars.value_counts(ascending=True)

2     36085
1     38347
3     63586
4    124241
5    132556
Name: stars, dtype: int64

In [28]:
data_business = []
with open('F:\yelp\yelp_academic_dataset_business.json') as f:
    for line in f:
        data_business.append(json.loads(line))

In [29]:
df_business = pd.DataFrame(data_business)

In [30]:
business_merged_df = pd.merge(merged_df, df_business, on='business_id', how='inner')

In [31]:
business_merged_df.shape

(394815, 32)

In [32]:
business_merged_df.drop(['neighborhoods', 'latitude', 'longitude', 'elite', 'yelping_since', 'compliments', 'full_address', 'city', 'state', 'categories', 'open', 'hours', 'attributes'], axis=1, inplace=True)

In [33]:
business_merged_df.head(n=1)

,business_id,date,review_id,stars_x,text,type_x,user_id,votes_x,average_stars,fans,friends,name_x,review_count_x,type_y,votes_y,name_y,review_count_y,stars_y,type
0,UsFtqoBl7naz8AVUBZMjQQ,2013-11-08,Di3exaUCFNw1V4kSNW5pgA,5,All the food is great here. But the best thing...,review,uK8tzraOp4M5u3uYrqIBXg,"{u'funny': 0, u'useful': 0, u'cool': 0}",3.84,2,"[53wSMIztehsjP-vJZBS54A, CXhOU5apSRSTtEUiyoqH3Q]",Robin,46,user,"{u'funny': 2, u'useful': 36, u'cool': 5}",Clancy's Pub,4,3.5,business


In [38]:
for index, row in business_merged_df.head(n=5).iterrows():
    if len(row['friends']) != 0:
        print index

0
1
2
3
4


In [42]:
for index, row in business_merged_df.head(n=5).iterrows():
    print index

0
1
2
3
4


In [43]:
%timeit business_sorted_df = business_merged_df.sort_index(by=['business_id'], ascending=[True])

1 loops, best of 3: 1.5 s per loop


F:\PythonIDE\lib\site-packages\ipykernel\__main__.py:257: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [61]:
#1 is b-id, 3 is review id, 4 is stars_x, 7 is user id, 11 friend ids, 18 is business rating
for row in business_merged_df.head(n=1).itertuples():
    print "The business: " + str(row[1]) + " has an avg rating of: " + str(row[18])
    print "User " + str(row[7]) +" rated " + str(row[1]) + " with " + str(row[4]) + " stars. "

The business: UsFtqoBl7naz8AVUBZMjQQ has an avg rating of: 3.5
User uK8tzraOp4M5u3uYrqIBXg rated UsFtqoBl7naz8AVUBZMjQQ with 5 stars. 


In [1]:
new_df = pd.DataFrame()
for row in business_merged_df.head(n=10).itertuples():
    if len(row['friends']) != 0:
        business_rating = row['stars_y']
        business_id = row['business_id']
        user_id = row['user_id']
        print "The business: " + str(business_id) + " has an avg rating of: " + str(business_rating)  
        user_rating = row['stars_x']
        print "User " + str(row['user_id']) +" rated " + str(business_id) + " with " + str(user_rating) + " stars. "
        count = 0
        friends = 0
        friends_who_rated = 0
        for fid in row['friends']:
#             if row['user_id'] == fid:
            friends += 1
            if any(business_merged_df[business_merged_df['user_id'] == fid]):
                for idx in business_merged_df[business_merged_df['user_id'] == fid].index:
#                    print business_merged_df.ix[idx]['user_id'] + "  =  " + fid   ##prints dat at idx, user & friend id r same
#                    print str(business_merged_df.ix[idx]['business_id']) + " at index "
                    if business_merged_df.ix[idx]['business_id'] == business_id:
                        #print "Friend " + business_merged_df.ix[idx]['user_id'] + " rated " + business_merged_df.ix[idx]['business_id']
                        #print "these many stars: " + business_merged_df.ix[idx]['stars_x'].astype(str)
                        count += business_merged_df.ix[idx]['stars_x']
                        friends_who_rated += 1
        #print "Sum of total ratings given by friends: " + str(count)
        #print "Total friends: " + str(friends)
        #print "Total friends who rated the business: " + str(friends_who_rated)
        if friends_who_rated != 0:
            average = float(count/float(friends_who_rated))
            #print "Friends average: " + str(average)
            data = pd.DataFrame({"business_id": [business_id], "business_stars": [business_rating], "user_stars": [user_rating], 
                                 "friends_avg_rating": [average], "user_id": [user_id], "friends_who_rated": [friends_who_rated],
                                 "total_friends": [friends]})
            new_df = new_df.append(data, ignore_index=True)
#                     else:
#                         print "no"
#             if any(business_merged_df['user_id'] == fid):
#                 print fid + " = " + business_merged_df['user_id']
#                 count += row['stars_x']
#         print "total friends' votes: %s" % count
#         print "total friends: %s" % friends
#                 count += row['stars_x']
#             print count


NameError: name 'pd' is not defined

In [ ]:
new_df